# fetch data files 

In [1]:
from bs4 import BeautifulSoup
import chardet
import datetime
import duckdb
import io
import pandas as pd
import requests

In [2]:
data_path = '../data/'
country_list = 'config/country_url.csv'

In [3]:
countries = pd.read_csv(data_path+country_list)['country']
countries

0         england
1        scotland
2         germany
3           italy
4           spain
5          france
6     netherlands
7         belgium
8        portugal
9          turkey
10         greece
11      argentina
12        austria
13         brazil
14          china
15        denmark
16        finland
17        ireland
18          japan
19         mexico
20         norway
21         poland
22        romania
23         russia
24         sweden
25    switzerland
26            usa
Name: country, dtype: object

In [5]:

def read_problematic_csv(url):
    # fetch file content of the linked csv file
    response = requests.get(url)
    content = response.content
    
    # detect the encoding
    detected = chardet.detect(content)
    encodings_to_try = [
        detected['encoding'],  # first attempt with detected encoding
        'windows-1252',        # maybe windows encoding?
        'latin-1',             # a permissive encoding
        'utf-8'                # standard, preferred encoding
    ]
    
    # remove duplicates, remove 'None's
    encodings_to_try = list(set([e for e in encodings_to_try if e]))
    
    # try each encoding 
    for encoding in encodings_to_try:
        try:
            csv_content = content.decode(encoding)
            # determine the number of columns from the header
            first_line = csv_content.split('\n')[0]
            header_count = first_line.count(',') + 1
            
            # Read the CSV with pandas, specifying the number of columns
            df = pd.read_csv(
                io.StringIO(csv_content),
                usecols=range(header_count),
                parse_dates=['Date'],
                dayfirst=True,
                on_bad_lines='warn',
                encoding_errors='replace'  # replace characters that can't be decoded
            )
            
            # print(f"successfully read with encoding: {encoding}")
            return df
            
        except Exception as e:
            print(f"reading/parsing csv file content failed with encoding {encoding}: {str(e)}")
            continue
    
    # If we've tried all encodings and none worked
    raise ValueError("could not read the csv file with any encoding")
# test
# url =datafile_url
# df = read_problematic_csv(datafile_url)
# print(df.head())

In [6]:
country=countries[5]
datafile_urls = pd.read_csv(data_path+'config/'+country+'_data_file_urls.csv')
print(country, 'has', len(datafile_urls), 'urls')
datafile_url=datafile_urls['data_url'][8]
# https://www.football-data.co.uk/mmz4281/9900/F1.csv
print('reading', datafile_url)
df = pd.read_csv(datafile_url, parse_dates=['Date'], dayfirst=True)
df = df.dropna(axis=1,how='all')
df['country'] = country
df['url'] = datafile_url
filename = \
    country                           \
    + '_'                             \
    + datafile_url                    \
    .split('football-data.co.uk/')[1] \
    .replace('/', '_')                \
    .split('.')[0]                    \
    + '.csv'                          \
    .lower()

print('filename=', filename)

france has 61 urls
reading https://www.football-data.co.uk/mmz4281/9900/F1.csv
filename= france_mmz4281_9900_F1.csv


/var/folders/xn/jj94n4tj5w90v1m3njdf44n80000gp/T/ipykernel_98487/744390444.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(datafile_url, parse_dates=['Date'], dayfirst=True)


In [7]:
for country in countries:
    datafile_urls = pd.read_csv(data_path+'config/'+country+'_data_file_urls.csv')
    print(country, 'has', len(datafile_urls), 'urls')
    for datafile_url in datafile_urls['data_url']:
        print('reading', datafile_url)
        df = read_problematic_csv(datafile_url)
        # df = pd.read_csv(datafile_url, parse_dates=['Date'], dayfirst=True, on_bad_lines='error')
        df['country'] = country
        filename =                                \
            data_path                             \
            + 'raw/'                              \
            + country                             \
            + '_'                                 \
            + datafile_url                        \
                .split('football-data.co.uk/')[1] \
                .replace('/', '_')                \
                .split('.')[0]                    \
            + '.csv'                              \
            .lower()
        df.to_csv(filename)

england has 148 urls
reading https://www.football-data.co.uk/mmz4281/1718/E3.csv
reading https://www.football-data.co.uk/mmz4281/0102/E3.csv


/var/folders/xn/jj94n4tj5w90v1m3njdf44n80000gp/T/ipykernel_98487/1440073136.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


reading https://www.football-data.co.uk/mmz4281/1415/E0.csv


/var/folders/xn/jj94n4tj5w90v1m3njdf44n80000gp/T/ipykernel_98487/1440073136.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


reading https://www.football-data.co.uk/mmz4281/0708/E0.csv


...

reading https://www.football-data.co.uk/mmz4281/2223/G1.csv
reading https://www.football-data.co.uk/mmz4281/9495/G1.csv
reading https://www.football-data.co.uk/mmz4281/0102/G1.csv


/var/folders/xn/jj94n4tj5w90v1m3njdf44n80000gp/T/ipykernel_98487/1440073136.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(
/var/folders/xn/jj94n4tj5w90v1m3njdf44n80000gp/T/ipykernel_98487/1440073136.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


reading https://www.football-data.co.uk/mmz4281/9697/G1.csv
reading https://www.football-data.co.uk/mmz4281/0304/G1.csv


/var/folders/xn/jj94n4tj5w90v1m3njdf44n80000gp/T/ipykernel_98487/1440073136.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(
/var/folders/xn/jj94n4tj5w90v1m3njdf44n80000gp/T/ipykernel_98487/1440073136.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


argentina has 1 urls
reading https://www.football-data.co.uk/new/ARG.csv
austria has 1 urls
reading https://www.football-data.co.uk/new/AUT.csv
brazil has 1 urls
reading https://www.football-data.co.uk/new/BRA.csv
china has 1 urls
reading https://www.football-data.co.uk/new/CHN.csv
denmark has 1 urls
reading https://www.football-data.co.uk/new/DNK.csv
finland has 1 urls
reading https://www.football-data.co.uk/new/FIN.csv
ireland has 1 urls
reading https://www.football-data.co.uk/new/IRL.csv
japan has 1 urls
reading https://www.football-data.co.uk/new/JPN.csv
mexico has 1 urls
reading https://www.football-data.co.uk/new/MEX.csv
norway has 1 urls
reading https://www.football-data.co.uk/new/NOR.csv
poland has 1 urls
reading https://www.football-data.co.uk/new/POL.csv
romania has 1 urls
reading https://www.football-data.co.uk/new/ROU.csv
russia has 1 urls
reading https://www.football-data.co.uk/new/RUS.csv
sweden has 1 urls
reading https://www.football-data.co.uk/new/SWE.csv
switzerland ha

In [ ]:
datafile_url